In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F
import torchvision as tv
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import hamming_loss, accuracy_score, f1_score, precision_score, recall_score
from skmultilearn.dataset import load_dataset

from C2AE import save_model, eval_metrics, load_model

In [2]:
device = torch.device('cuda')
train_x, train_y, feat_names, label_names = load_dataset('tmc2007_500', 'train')
test_x, test_y, _, _ = load_dataset('tmc2007_500', 'test')

train_dataset = TensorDataset(torch.tensor(train_x.todense(), device=device, dtype=torch.float),torch.tensor(train_y.todense(), device=device,dtype=torch.float))
test_dataset = TensorDataset(torch.tensor(test_x.todense(), device=device, dtype=torch.float), torch.tensor(test_y.todense(), device=device, dtype=torch.float))

tmc2007_500:train - exists, not redownloading
tmc2007_500:test - exists, not redownloading


# Defining metrics.

In [3]:
def micro_r(y_t, y_p):
    return recall_score(y_t, y_p, average='micro')
def macro_r(y_t, y_p):
    return recall_score(y_t, y_p, average='macro')
def micro_p(y_t, y_p):
    return precision_score(y_t, y_p, average='micro')
def macro_p(y_t, y_p):
    return precision_score(y_t, y_p, average='macro')
def micro_f1(y_t, y_p):
    return f1_score(y_t, y_p, average='micro')
def macro_f1(y_t, y_p):
    return f1_score(y_t, y_p, average='macro')
def ham_los(*args, **kwargs):
    return hamming_loss(*args, **kwargs)

In [5]:
class TMCModel(torch.nn.Module):
    
    def __init__(self):
        super(TMCModel, self).__init__()
        self.fc1 = torch.nn.Linear(500, 22)
    
    def forward(self, x):
        x = self.fc1(x)
        return x

# Making TensorDatasets

In [4]:
len(train_dataset), len(test_dataset)

(21519, 7077)

In [6]:
num_epochs = 1000
lr = 0.0001
batch_size=32

net = TMCModel().to(device)
writer = SummaryWriter(comment='tmc_fc')
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
# train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, num_workers=2)

In [7]:
train_loss = []
test_loss = []

for epoch in range(num_epochs+1):
    
    net.train()
    loss_tracker = 0.0
    for x, y in train_dataloader:
        optimizer.zero_grad()
        preds = net(x)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        loss_tracker+=loss.item()
    train_loss.append(loss_tracker/len(train_dataloader))
    writer.add_scalar('train/loss', loss_tracker/len(train_dataloader), epoch)
    
    if epoch % 5 == 0:
        net.eval()
        test_tracker = 0.0
        for x, y in test_dataloader:
            preds = net(x)
            loss = criterion(preds, y)
            test_tracker += loss.item()
        test_loss.append(test_tracker/len(test_dataloader))
        writer.add_scalar('val/loss', test_tracker/len(test_dataloader), epoch)

        # Log all the good metrics to the board.
        mets = eval_metrics(net, [ham_los, accuracy_score, micro_f1, micro_p, micro_r, macro_f1, macro_p, macro_r], 
                                [test_dataset, train_dataset], device, apply_sig=True)
        for k, v in mets['dataset_1'].items():
            writer.add_scalar(f'train/{k}', v, epoch)
        # Train
        for k, v in mets['dataset_0'].items():
            writer.add_scalar(f'val/{k}', v, epoch)
        print("Epoch: {}, Train loss: {}, Test loss: {}".format(epoch, train_loss[-1], test_loss[-1]))
    torch.save(net.state_dict(), f'./models/tmc_fc/{epoch}.pt')

C:\Users\Stone\miniconda3\envs\torch_env\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 0, Train loss: 0.5197060941587054, Test loss: 0.41104362462017985
Epoch: 5, Train loss: 0.23351587344066171, Test loss: 0.22873962026190114
Epoch: 10, Train loss: 0.1979138252462356, Test loss: 0.19559759304330154
Epoch: 15, Train loss: 0.18074242742995097, Test loss: 0.17922576530291154
Epoch: 20, Train loss: 0.17030092164940502, Test loss: 0.16914177075162665
Epoch: 25, Train loss: 0.16324709213019126, Test loss: 0.1622517042466112
Epoch: 30, Train loss: 0.15815647709201386, Test loss: 0.1572293255914439
Epoch: 35, Train loss: 0.1543061088820446, Test loss: 0.15339991487227045
Epoch: 40, Train loss: 0.1512896227823324, Test loss: 0.15038158398893503
Epoch: 45, Train loss: 0.1488605431249202, Test loss: 0.14794019874822986
Epoch: 50, Train loss: 0.1468604507829778, Test loss: 0.1459233756135176
Epoch: 55, Train loss: 0.14518305993629246, Test loss: 0.14422777543465296
Epoch: 60, Train loss: 0.1437545874741421, Test loss: 0.14278110381853473
Epoch: 65, Train loss: 0.142522311748

# Save the model after training.

In [50]:
def save_model(model, path):
    torch.save(model.state_dict(), path)
def load_model(model_cls, path, *args, **kwargs):
    model = model_cls(*args, **kwargs)
    model.load_state_dict(torch.load(path))
    model.eval()
    return model

### Picking best model

In [11]:
eval_net = load_model(TMCModel, './models/tmc_fc/1000.pt').to(device)

In [12]:
mets = eval_metrics(eval_net, [ham_los, accuracy_score, micro_f1, micro_p, micro_r, macro_f1, macro_p, macro_r], [test_dataset, train_dataset], device, apply_sig=True)
mets

{'dataset_0': {'ham_los': 0.05343815432836204,
  'accuracy_score': 0.3300833686590363,
  'micro_f1': 0.7169490372184798,
  'micro_p': 0.7627768930070943,
  'micro_r': 0.6763157894736842,
  'macro_f1': 0.6546244349804987,
  'macro_p': 0.7888285445446485,
  'macro_r': 0.5790272326789471},
 'dataset_1': {'ham_los': 0.052674380779776014,
  'accuracy_score': 0.3398392118592871,
  'micro_f1': 0.7241756904732937,
  'micro_p': 0.7699508431921349,
  'micro_r': 0.6835379604109246,
  'macro_f1': 0.6553832344888076,
  'macro_p': 0.7876056752223366,
  'macro_r': 0.5791569417923896}}